<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Bubble_Dew_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [16]:
from dotmap import DotMap
from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32 bit - enable 64 bit - double precision

from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

from tools.tree_array_transform2 import VSC, Comp, Range, RangeArray
import tools.che as che
R=8.314 # J/(mol K)

In [17]:
p = che.Props(['Methanol', 'Isopropanol'])

In [18]:
def bubble_T(c):
    return jnp.sum(c.x * p.Pvap(c.T))-c.P

In [19]:
c=DotMap()
c.x = jnp.array([0.5,0.5])
c.T = Range(300., 273., 400.)
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, bubble_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

345.3145905085819


In [20]:
def dew_T(c):
    return jnp.sum(c.y * c.P / p.Pvap(c.T))-1.

c=DotMap()
c.y = jnp.array([0.5,0.5])
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, dew_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

348.42605578172765


In [21]:
c = DotMap()
c.T = Range(300, 273.15, 400.)
P=101325
molfracs = jnp.linspace(0.,1.,11)

def solve_bubble_T(x1, P):
  c.x = jnp.array([x1, 1-x1])
  c.P = P
  vsc = VSC(c, bubble_T)
  vsc.solve(verbosity=0)
  return (vsc.v.T)

bubble_T_array = jnp.array([solve_bubble_T(x1, P) for x1 in molfracs])

In [22]:
bubble_T_array

DeviceArray([355.62053871, 353.28376619, 351.09848853, 349.05018915,
             347.12601929, 345.31459051, 343.60579324, 341.99063836,
             340.46111868, 339.01008816, 337.63115666], dtype=float64)

In [23]:
def solve_dew_T(y1, P):
  c.y = jnp.array([y1, 1-y1])
  c.P = P
  vsc = VSC(c, dew_T)
  vsc.solve(verbosity=0)
  return (vsc.v.T)

dew_T_array = jnp.array([solve_dew_T(y1, P) for y1 in molfracs])

In [24]:
dew_T_array

DeviceArray([355.62053871, 354.34764878, 353.00296829, 351.57654096,
             350.05606304, 348.42605578, 346.66662825, 344.7515556 ,
             342.64514834, 340.29684768, 337.63115666], dtype=float64)

In [25]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=molfracs, y=bubble_T_array, name='Liq', mode='lines')
fig.add_scatter(x=molfracs, y=dew_T_array, name='Vap', mode='lines')
fig.update_xaxes(showline=True, linewidth=1, title_text='x1, y1')
fig.update_yaxes(showline=True, linewidth=1, title_text='Temperature (K)')
fig.update_layout(width=600,height=400, title_text=f'MeOH / iPrOH ({P:.0f} Pa)', title_x=0.5)

In [26]:
molfracs = jnp.linspace(0.,1.,6)
Ps = jnp.linspace(100000,20000, 2)


In [27]:
bubble_T_array = jnp.array([[solve_bubble_T(x1, P) for x1 in molfracs] for P in Ps])
dew_T_array = jnp.array([[solve_dew_T(x1, P) for x1 in molfracs] for P in Ps])

In [37]:
fig=make_subplots(rows=1,cols=1)

x,y = jnp.meshgrid(molfracs, Ps)

colorscale = [[0, 'rgba(0,0,255,1)'], 
              [1, 'rgba(0,255,0,1)']]

fig.add_surface(z=bubble_T_array, x=x, y=y, name='Liq', surfacecolor=jnp.full_like(bubble_T_array,0), cmin=0, cmax=1, colorscale=colorscale, showscale=False)
fig.add_surface(z=dew_T_array, x=x, y=y, name='Vap', surfacecolor=jnp.full_like(bubble_T_array,1), cmin=0, cmax=1, colorscale=colorscale, showscale=False)
fig.update_layout(scene = dict(
                    xaxis_title='x1, y1',
                    yaxis_title='Pressure (Pa)',
                    zaxis_title='T (K)'),
                    height = 700, width=700)
